# Завдання 1. Перевірка унікальності паролів за допомогою фільтра Блума

In [1]:
import hashlib


In [2]:
class BloomFilter:
    def __init__(self, size: int, num_hashes: int):
        self.size = size
        self.num_hashes = num_hashes
        self.bit_array = [0] * size

    def _hashes(self, item: str):
        """Генерує num_hashes хешів для рядка item"""
        hashes = []
        for i in range(self.num_hashes):
            hash_input = f"{item}{i}".encode()
            hash_digest = hashlib.md5(hash_input).hexdigest()
            hash_int = int(hash_digest, 16)
            hashes.append(hash_int % self.size)
        return hashes

    def add(self, item: str):
        """Додає елемент до фільтра"""
        for h in self._hashes(item):
            self.bit_array[h] = 1

    def contains(self, item: str) -> bool:
        """Перевіряє, чи може елемент бути в фільтрі"""
        return all(self.bit_array[h] for h in self._hashes(item))


In [3]:
def check_password_uniqueness(bloom: BloomFilter, passwords: list) -> dict:
    results = {}

    for pwd in passwords:
        if not isinstance(pwd, str) or pwd.strip() == "":
            results[pwd] = "некоректний пароль"
            continue

        if bloom.contains(pwd):
            results[pwd] = "вже використаний"
        else:
            results[pwd] = "унікальний"
            bloom.add(pwd)

    return results


In [4]:
if __name__ == "__main__":
    # Ініціалізація фільтра Блума
    bloom = BloomFilter(size=1000, num_hashes=3)

    # Додавання існуючих паролів
    existing_passwords = ["password123", "admin123", "qwerty123"]
    for password in existing_passwords:
        bloom.add(password)

    # Перевірка нових паролів
    new_passwords_to_check = ["password123", "newpassword", "admin123", "guest", "", None]
    results = check_password_uniqueness(bloom, new_passwords_to_check)

    # Виведення результатів
    for password, status in results.items():
        print(f"Пароль '{password}' — {status}.")


Пароль 'password123' — вже використаний.
Пароль 'newpassword' — унікальний.
Пароль 'admin123' — вже використаний.
Пароль 'guest' — унікальний.
Пароль '' — некоректний пароль.
Пароль 'None' — некоректний пароль.


# Завдання 2. Порівняння продуктивності HyperLogLog із точним підрахунком унікальних елементів

In [5]:
!pip install datasketch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 2.2 MB/s eta 0:00:00


In [6]:
import time
from datasketch import HyperLogLog
import re
import pandas as pd


In [7]:
def load_ip_addresses(filepath):
    ip_pattern = re.compile(r'\b(?:\d{1,3}\.){3}\d{1,3}\b')
    ip_list = []

    with open(filepath, 'r') as file:
        for line in file:
            match = ip_pattern.search(line)
            if match:
                ip_list.append(match.group())

    return ip_list


In [8]:
def count_unique_ips_exact(ip_list):
    start = time.time()
    unique_ips = set(ip_list)
    duration = time.time() - start
    return len(unique_ips), duration


In [9]:
def count_unique_ips_hll(ip_list, hll_precision=14):
    hll = HyperLogLog(p=hll_precision)
    start = time.time()
    for ip in ip_list:
        hll.update(ip.encode('utf8'))
    duration = time.time() - start
    return hll.count(), duration


In [10]:
# Вказуємо шлях до файлу
file_path = "/content/sample_data/lms-stage-access.log"

# Завантажуємо IP-адреси
ip_addresses = load_ip_addresses(file_path)

# Підрахунок точний
exact_count, exact_time = count_unique_ips_exact(ip_addresses)

# Підрахунок через HyperLogLog
hll_count, hll_time = count_unique_ips_hll(ip_addresses)

# Виведення таблиці порівняння
results_df = pd.DataFrame({
    "Метод": ["Точний підрахунок", "HyperLogLog"],
    "Унікальні елементи": [exact_count, hll_count],
    "Час виконання (сек)": [round(exact_time, 4), round(hll_time, 4)]
})

import IPython.display as disp
disp.display(results_df)


,Метод,Унікальні елементи,Час виконання (сек)
0,Точний підрахунок,28.000000,0.0022
1,HyperLogLog,28.023953,0.1130


In [11]:
abs_error = abs(exact_count - hll_count)
rel_error = abs_error / exact_count * 100
print(f"Абсолютна похибка: {abs_error}, Відносна: {rel_error:.2f}%")


Абсолютна похибка: 0.02395307542871805, Відносна: 0.09%
